# Start the server

In [ ]:
# only uncomment & run this cell if the next one fails
# !sudo apt install --no-install-recommends google-perftools -y

In [ ]:
import subprocess
import os

# save the current working directory
cwd = os.getcwd()

# change the current working directory
os.chdir('../stable-diffusion-webui')

subprocess.Popen(["bash", "-c", "wget -qO- https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/webui.sh | bash"])

# switch back to the original working directory
os.chdir(cwd)


In [ ]:
!pip install requests beautifulsoup4 tqdm

# Create a deployment URL

Do this from the console. Here's a link to the docs: https://brev.dev/docs/how-to/deployments. The port is 7860

# Helpful functions to load new models (just run this)

In [ ]:
import os
from urllib.parse import urljoin, urlparse
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

def download_file(url, save_dir):
    # Send a GET request
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Look for the download button
    download_button = soup.find("a", {"class": "mantine-UnstyledButton-root mantine-Button-root mantine-zj7kjy"})

    # Get the download link
    download_link = urljoin(url, download_button.get('href'))

    # Try to extract the filename from the Content-Disposition header
    content_disposition = response.headers.get('content-disposition', '')
    filenames = re.findall('filename=(.+)', content_disposition)

    # If the filename is found in the Content-Disposition header, use it;
    # otherwise, generate the filename from the URL
    if filenames:
        filename = filenames[0]
    else:
        parsed_url = urlparse(url)
        filename = os.path.split(parsed_url.path)[-1] + ".safetensors"

    # Make sure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Download the file and save it
    response = requests.get(download_link, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024 # 1 Kibibyte
        t=tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(os.path.join(save_dir, filename), 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
        if total_size != 0 and t.n != total_size:
            print("ERROR, something went wrong.")
        print(f"File downloaded successfully at {os.path.join(save_dir, filename)}")
    else:
        print("Failed to download the file")


url = 'https://civitai.com/models/4384/dreamshaper'
save_dir = '/home/ubuntu/stable-diffusion-webui/models/Stable-diffusion'
download_file(url, save_dir)


# Load New Models (works with CivitAI)

In [ ]:
# url for the civit ai model
url = 'https://civitai.com/models/99020/touch-the-grass'

# save models in this directory to use them with AUTOMATIC1111 
save_dir = '/home/ubuntu/stable-diffusion-webui/models/Stable-diffusion'

# run the function to fetch the model
download_file(url, save_dir)